# Training Topic Model (LDA) on the Airbnb Comments

In [103]:

# import and setup modules we'll be using in this notebook
import logging
import itertools
import pandas as pd
import numpy as np
import gensim
import sys
reload(sys)
import cPickle
import re
sys.setdefaultencoding('utf8')
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [104]:
from gensim.utils import smart_open, simple_preprocess
from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS

def tokenize(text):
    return [token for token in simple_preprocess(text) if token.lower() not in STOPWORDS]

def iter_token(id_,comment):
    """Yield each article from the Wikipedia dump, as a `(title, tokens)` 2-tuple."""
  #  ignore_namespaces = 'Wikipedia Category File Portal Template MediaWiki User Help Book Draft'.split()
    for id_,comment in zip(id_,comment):
        tokens=tokenize(comment)
       # if len(tokens) < 50 :
        #    continue  # ignore short articles and various meta-articles
        yield id_, tokens
def person_remover(comment,person):
    return comment.replace(person,'')    

# Reading in data

In [105]:
col_to_read = ['listing_id','id','lan']
df_all_reviews= pd.read_csv('../DataFiles/merged_reviews_withCitiesLan.csv',usecols=col_to_read)

df_person = pd.read_csv('../DataFiles/sentences_withPersonName_NYC_.csv',header=None,sep='\t')
df_person.columns=['no','year','city','id','person','comment']

df_person=df_person.dropna()
df_person.year=df_person.year.astype(int)
df_person.id=df_person.id.astype(int)
df=pd.merge(df_person,df_all_reviews,on='id',how='left')
df=df[df.lan=='en']
df.listing_id.nunique()

27247

In [106]:
df['comment'] = df.apply(lambda row: person_remover(row['comment'], row['person']), axis=1)

df_text=df.groupby(['listing_id','year'])['comment'].apply(lambda x: '.'.join(x)).reset_index()
df_text=df_text.sort_values(['listing_id','year'], ascending=True)




In [107]:
#df_text=df_text[df_text.year>2009]
#df_=pd.DataFrame(df_text.groupby(['listing_id']).year.count())
#indices=df_[df_.year==7].index.tolist()
#df_text=df_text[df_text['listing_id'].isin(indices)]

In [108]:
df_text.to_csv('../DataFiles/sentences_withPersonName_NYC_listing.csv')

# Tokenizing documents

In [109]:
# Tokenize the documents.

from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
docs = list(df_text.comment)
for idx in range(len(df_text.comment)):
    docs[idx]=re.sub(r'[^\x00-\x7F]+',' ', docs[idx])
    docs[idx] = docs[idx].lower()# Convert to lowercase.
    docs[idx] =tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]


# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [110]:
from nltk.stem.wordnet import WordNetLemmatizer


            
# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [111]:
# Compute bigrams.

from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [112]:
# Remove rare and common tokens.

from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)
dictionary.save('nyc_lda.dic')

with open(r"docs.pickle", "wb") as output_file:
    cPickle.dump(docs, output_file)

# Vectorizing documents, and save the corpus

In [113]:

# Vectorize data.

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

with open(r"corpus.pickle", "wb") as output_file:
    cPickle.dump(corpus, output_file)

In [114]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# Train and Save the LDA model

In [115]:
# Train LDA model.

from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

model.save('lda.pkl')

# High frequent words in each topic

In [116]:
model = gensim.models.LdaModel.load('lda.pkl')
#top_topics = model.top_topics(corpus, num_words=5)
top_topics=model.show_topics()
# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
#avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
#print('Average topic coherence: %.4f.' % avg_topic_coherence)

topics=[]
top_topics=model.show_topics()
for t in top_topics:
    topic={}
    for chunk in t[1].split('+'):
        topic[chunk.split('*')[1].strip("'")] = float(chunk.split('*')[0])
    topics.append(topic)
    
from pprint import pprint
#pprint(top_topics)
df_list=[]

for topic in topics:

    df_= pd.DataFrame([(str(k),v) for k,v in topic.iteritems() ])
    df_.sort_values([1], ascending=False)
    
    df_list.append( df_[0])

df = pd.concat(df_list, axis = 1)
df.columns=[ [ 'Topic'+str(i) for i in range(len(df_list))]]
df

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9
0,"""let""","""recommendation""","""brooklyn""","""question""","""floor""","""sheet""","""west""","""nyc""","""good""","""walk"""
1,"""day""","""information""","""husband""","""quick""","""issue""","""fresh""","""pick""","""welcomed""","""time""","""manhattan"""
2,"""arrived""","""eat""","""lovely""","""meet""","""problem""","""fridge""","""easy_communicate""","""dog""","""helpful""","""station"""
3,"""late""","""neighborhood""","""home""","""helpful""","""building""","""coffee""","""east""","""home""","""wonderful""","""block"""
4,"""arrival""","""gave""","""wonderful""","""accommodating""","""little""","""breakfast""","""east_village""","""welcome""","""recommend""","""minute"""
5,"""check""","""bar""","""family""","""check""","""wasn""","""towel""","""village""","""feel_welcome""","""welcoming""","""subway"""
6,"""hour""","""area""","""loved""","""needed""","""air""","""tea""","""coordinate""","""felt""","""room""","""train"""
7,"""met""","""local""","""house""","""responsive""","""said""","""water""","""easy""","""cat""","""nice""","""located"""
8,"""flight""","""restaurant""","""beautiful""","""easy""","""door""","""hidden""","""communicate""","""feel""","""staying""","""location"""
9,"""early""","""provided""","""loft""","""communication""","""bit""","""provided""","""key""","""like""","""friendly""","""close"""


In [117]:
model.get_document_topics(corpus[0])

[(0, 0.055415622389523231),
 (1, 0.04706077545438378),
 (2, 0.061441294541481693),
 (3, 0.20849311155273964),
 (4, 0.051737180681578877),
 (5, 0.012986750453466487),
 (6, 0.015140501839861391),
 (7, 0.04770341642886089),
 (8, 0.46159754078241311),
 (9, 0.038423805875690956)]